# Duck typing

The following function `sum_all` sums numbers of a list.

In [ ]:
def sum_all(lst):
    ret = None
    for elem in lst:
        if ret is None:
            ret = elem
        else:
            ret = ret + elem
    return ret
    
    
print(sum_all([1,2,3]))

But wait, `lst` need not be a list and the elements of `lst` need not be numbers. "Sums numbers of a list" does not fully describe the capability of `sum_all`. 

Really, you can use `sum_all(lst)` if you can iterate through the elements of `lst` with a for loop (i.e., `lst` is an "iterable" as we define later) and you can use `+` with the elements of `lst` (i.e., the elements of `lst` are objects with the `__add__` method).

In [ ]:
lst1 = ['Python was named after ', 'the British TV series "Monty Python." ']
lst2 = ['The Dutch creator of Python, Guido van Rossum, seems to have a British sense of humor.']
print(sum_all(lst1+lst2))  # list of strings

c1 = Complex(1,2)
c2 = Complex(3,4)
c3 = Complex(-5,0)

print(sum_all((c1,c2,c3)))   # tuple of Complex

In the context of logic (논리학), the following saying describes a form of abductive reasoning:

> "If it looks like a duck, swims like a duck, and quacks like a duck, then it probably is a duck."

In the context of programming, __duck typing__ refers to the practice of caring about what the object can do, rather than what it is.

Consider the following implementation of gradient descent for
$$
\begin{array}{ll}
\underset{x\in\mathbb{R}^n}{\mbox{minimize}}&
\frac{1}{2}\|Ax-b\|^2
\end{array}
$$

In [ ]:
#A = m x n matrix
b = np.array(m)

x = np.zeros(n)
for _ in range(10000) :
    x = x - alpha*A.T@(huber_grad(A@x-b))

What must `A` be able to __do__? (Note, we are not asking what `A` __is__.)

- `A` must have `__matmul__(self, np_vector)` must be implemented so that `A@x` with a numpy vector `x` is allowed.
- `A` must have instance variable `T` so that `A.T` is allowed.

There are cases where you can implement matrix-vector multiplication with $A$ and $A^T$, but forming the $m\times n$ matrix is inefficient (e.g. sparse matrix, FFT, and convolution). In these cases, you can provide objects supporting matrix-vector products without directly forming the full numpy matrix.

Duck typing is Pythonic. In strongly typed languages like C++ and Java, duck typing is mostly impossible, and you are required to use inheritance or function pointers to achieve similar effects.

In [ ]:
import numpy as np

'''
1D convolution operator problem starter code
@author: Ji Sun Park and Ernest K. Ryu
'''

class Convolution1d :
    
    ''' Inite object with filter'''
    def __init__(self, filt) :
        self.__filt = filt
        self.__r = filt.size
        self.T = TransposedConvolution1d(self.__filt)

    '''
    Convolution operation
        Usage : self @ vector
    '''
    def __matmul__(self, vector) :
        r, n = self.__r, vector.size
        
        return np.asarray([sum(self.__filt * vector[j:j+r]) for j in range(n-r+1)])
    
'''
Transpose of 1-dimensional convolution operator used for the 
transpose-convolution operation A.T@(...)
'''
class TransposedConvolution1d :
    
    ''' Init TransposedConvolution1d object with filter '''
    def __init__(self, filt) :
        self.__filt = filt
        self.__r = filt.size

    '''
    Convolution operation
        Usage : A @ vector
    '''
    def __matmul__(self, vector) :
        r = self.__r
        n = vector.size + r - 1

        return np.asarray([sum(np.flip(self.__filt)[max(0,r-j-1):min(n-j,r)] * vector[max(0,j-r+1):min(j+1,n-r+1)]) for j in range(n)])




# Test Code
def huber_loss(x) :
    return np.sum( (1/2)*(x**2)*(np.abs(x)<=1) + (np.sign(x)*x-1/2)*(np.abs(x)>1) )
def huber_grad(x) :
    return x*(np.abs(x)<=1) + np.sign(x)*(np.abs(x)>1)


r, n, lam = 3, 20, 0.1

np.random.seed(0)
k = np.random.randn(r)
b = np.random.randn(n-r+1)
A = Convolution1d(k)
from scipy.linalg import circulant
A = circulant(np.concatenate((np.flip(k),np.zeros(n-r))))[2:,:]


x = np.zeros(n)
alpha = 0.01
for _ in range(100) :
    x = x - alpha*(A.T@(huber_grad(A@x-b))+lam*x)

print(huber_loss(A@x-b)+0.5*np.linalg.norm(x*x)**2)